In [4]:
from sklearn.neighbors import KDTree
import numpy as np
import random
import dask.array as da
import napari

%gui qt

coord_path = 'C:/Users/camer/OneDrive - Allen Institute/Documents/Unet/0539050069-0035_r.csv'

coord = np.loadtxt(coord_path, skiprows=1, delimiter=',')

In [5]:
zarr_img_path = 'C:/Users/camer/OneDrive - Allen Institute/Documents/Unet/0539050069-0035.zarr'
image = da.from_zarr(zarr_img_path)

In [6]:
viewer = napari.view_image(image, name='slice')
viewer.add_points(coord, size=5, face_color='red')

C:\Users\camer\anaconda3\envs\napari\lib\site-packages\napari\_vispy\vispy_image_layer.py:198: UserWarning: data shape (30000, 40000, 3) exceeds GL_MAX_TEXTURE_SIZE 16384 in at least one axis and will be downsampled. Rendering is currently in 2D mode.
  warnings.warn(


<Points layer 'coord' at 0x28c105c6d30>

C:\Users\camer\anaconda3\envs\napari\lib\site-packages\napari\_vispy\vispy_image_layer.py:198: UserWarning: data shape (30000, 40000, 3) exceeds GL_MAX_TEXTURE_SIZE 16384 in at least one axis and will be downsampled. Rendering is currently in 2D mode.
  warnings.warn(


In [275]:
kdt = KDTree(coord[:,:2], leaf_size=30, metric='euclidean')

close_points = []
close_set =[]
indices_to_drop = []

close = kdt.query_radius(coord[:,:2], r=3) 
for i in close:
    if i.size > 1:
        close_points.append(list(i))
for i in close_points:
    if not i in close_set:
        close_set.append(i)
print("Close points are: ", str(close_set))
for i in close_set:
    while len(i)>1:
        index = random.randrange(len(i))
        indices_to_drop.append(i[index])
        del i[index]

print(str(len(indices_to_drop)),"points to be removed:", str(indices_to_drop))

coord_clean = np.array([i for j, i in enumerate(coord) if j not in indices_to_drop])
coord_removed = np.array([i for j, i in enumerate(coord) if j in indices_to_drop])

print(str(coord.shape[0]-coord_clean.shape[0]), "points removed")

Close points are:  [[15172, 15173], [16419, 16420]]
2 points to be removed: [15173, 16420]
2 points removed


In [276]:
np.savetxt('/Users/elyse.morin/OneDrive - Allen Institute/Transgenic_Technology_Elyse/Projects/Unet_test_output/547656/csv/0539050069-0035_r_clean.csv', coord_clean, delimiter=",")

np.savetxt('/Users/elyse.morin/OneDrive - Allen Institute/Transgenic_Technology_Elyse/Projects/Unet_test_output/547656/csv/0539050069-0035_r_removed.csv', coord_removed, delimiter=",")